In [32]:
import requests
import random
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode
import os
import pandas as pd


In [33]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, ElementNotInteractableException, TimeoutException,JavascriptException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

-- chome_options that need to be run ** using

In [34]:
def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15")
    chrome_options.add_argument("--headless=new")  # New headless implementation
    chrome_options.add_argument("--disable-gpu")  # Disable GPU (useful for older environments)
    chrome_options.add_argument("--window-size=1920,1080")  # Set window size for headless
    chrome_options.add_argument("--no-sandbox")  # Bypass sandbox (useful in CI)
    chrome_options.add_argument("--disable-dev-shm-usage")  # Prevent shared memory issues
    return chrome_options

In [35]:
# driver = webdriver.Chrome(options=chrome_options)

-- function to get the page_navigation info ** using

In [36]:
def get_page_navigation_info(driver):
    try:
        # Find the page navigation bar
        page_navigation_bar = driver.find_element(By.XPATH, "//*[@class='listing -horizontal']")
        each_page = page_navigation_bar.find_elements(By.TAG_NAME, 'li')
        # page_amount = len(each_page)
        page_amount = 13 # hardcoding the page amount to investigate the page navigation bar
        print(f"there are {page_amount} total pages of cruises")
        return page_amount
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [37]:

'''
def update_url_fragment(driver, current_page):
    base_url = driver.current_url.split('#')[0]
    fragment = f"sort=featured:desc&page={str(current_page + 1)}&pageSize=10"
    new_url = f"{base_url}#{fragment}"
    return new_url
'''

'\ndef update_url_fragment(driver, current_page):\n    base_url = driver.current_url.split(\'#\')[0]\n    fragment = f"sort=featured:desc&page={str(current_page + 1)}&pageSize=10"\n    new_url = f"{base_url}#{fragment}"\n    return new_url\n'

-- extracting the urls from the page function , load urls to list ** using

In [38]:
def find_urls(driver):
    try:
        # Use JavaScript to find <a> tags within <li> elements and extract hrefs
        script = f"""
        const listItems = document.querySelectorAll('.c29_body_options_item');
        let hrefList = [];
        listItems.forEach(li => {{
            const anchor = li.querySelector('a'); // Search for <a> within each <li>
            if (anchor && anchor.href) {{
                hrefList.push(anchor.href);      // Push the href value to the list
            }}
        }});
        return hrefList;
        """
        
        # Execute the JavaScript and fetch the results
        href_list = driver.execute_script(script)
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return href_list

-- get_global_list function that loads the page and extracts urls ** using

In [39]:
def get_global_list():
    try:
        chrome_options = get_chrome_options()
        driver = webdriver.Chrome(options=chrome_options)
        url = "https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
        driver.get(url)
        time.sleep(10)
        page_amount = get_page_navigation_info(driver)
        # create global list to insert all the list into this one -- remeber to flatten at the end
        current_page = 1 # start at page 1
        global_list_urls = []
        
        while True:
            if current_page <= 2:
                print(f"You're currently on page: {current_page}, Total pages: {page_amount}")
                print(f"The current url is: {driver.current_url}")
                element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
                outer_text = element.get_attribute("outerText")
                is_page_loaded = outer_text != "0"
                
                if is_page_loaded:
                    print('Page loaded successfully')
                    print(f"verifying the cruises loaded: {element.get_attribute("outerText")}")
                    href_list = find_urls(driver)
                    global_list_urls.append(href_list)
                    print("the cruises hrefs on this page have been added to my list")
                    
                    # go to the next page
                    current_page += 1  # Increment the page number first
                    url = f"https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page={str(current_page)}&pageSize=10"
                    print(f"Navigating to new URL: {url}")
                    driver.get(url)
                    driver.refresh()
                    time.sleep(10)
                    

                else:
                    print(element.get_attribute("outerText"))
                    print('Page did not load successfully')
                    time.sleep(15)
            else:
                print("all pages have been scraped")
                break
        return global_list_urls

    except WebDriverException as e:
        print('Error loading page:', e)
        return None

    except ElementNotInteractableException as e:
        print('Element not interactable:', e)
        return None

    finally:
        # Close the browser
        driver.quit()

-- cleaning the values of the global list ** using

In [40]:

def cleaning_list(global_list_urls):
    # use nested loop as global list is a collection of lists
    # first round of cleaning
    for sublist in global_list_urls:
        i = 0
        while i < len(sublist):
            if sublist[i] == 'https://prod-aws.oceaniacruises.com/cruise-finder#':
                sublist.pop(i)
            else:
                i += 1
    
    # second round of cleaning
    for sublist in global_list_urls:
        i = 0
        while i < len(sublist):
            if "?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10" in sublist[i]:
                sublist[i] = sublist[i].replace("?sr=%2Fcruise-finder%23sort%3Dfeatured%3Adesc%26page%3D1%26pageSize%3D10", "")
            else:
                i += 1
    return global_list_urls

-- lets flatten out the global list ** using

In [41]:
def flatten_list(global_list_urls):
    list_of_list = [item for sublist in global_list_urls for item in sublist]
    return list_of_list

-- lets create the function to process each url - will be called at a later time ** using

In [42]:
def process_each_url(url):
    try:
        chrome_options = get_chrome_options()
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        time.sleep(10) 
        
        body_inner_html = driver.execute_script("return document.body.innerHTML;")
        driver.quit()
        soup = BeautifulSoup(body_inner_html, 'html5lib')
        map_soup = soup.find("div", class_ = "image").find("a")
        map_soup_ahref = map_soup['href']
        print(f"URL: {url}, Map URL: {map_soup_ahref}")
        return (url, map_soup_ahref)
    except Exception as e:
        print(f"An error occurred: {url},{e}")
        return None
        

-- lets pass the flatten_list to the scan url function ** using

In [43]:
def scan_list(list_of_list):
    map_href_list = []
    all_map_href_list = []
    for index, url in enumerate(list_of_list):
        map_href_list = process_each_url(url)
        percentage_left = (index + 1) / len(list_of_list) * 100
        print(f"Processed {percentage_left:.2f}% of the URL list")
        all_map_href_list.append(map_href_list)
    print(len(all_map_href_list))
    return all_map_href_list


-- downloading the global list to a text file onto my computer

In [44]:
def download_list(all_map_href_list):
    # define the downloads folder
    downloads_folder = '/Users/calvinpineda/Downloads'
    output_file_path = os.path.join(downloads_folder, "output-href-test-test-test.txt")

    # open the file in write mode
    with open(output_file_path, "w") as file:
        for url in all_map_href_list:
            file.write(f"{url}\n")

In [45]:
def master_function():
    global_list_urls = get_global_list()
    cleaning_list(global_list_urls) # modify the global list in place
    list_of_list = flatten_list(global_list_urls) # flatten the list
    all_map_href_list = scan_list(list_of_list) # scan the list
    download_list(all_map_href_list) # download the list
    print(all_map_href_list)
    print("done")
    

In [46]:
master_function()

there are 13 total pages of cruises
You're currently on page: 1, Total pages: 13
The current url is: https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10
Page loaded successfully
verifying the cruises loaded: VIEW 605 CRUISES
the cruises hrefs on this page have been added to my list
Navigating to new URL: https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=2&pageSize=10
You're currently on page: 2, Total pages: 13
The current url is: https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=2&pageSize=10
Page loaded successfully
verifying the cruises loaded: VIEW 605 CRUISES
the cruises hrefs on this page have been added to my list
Navigating to new URL: https://prod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=3&pageSize=10


KeyboardInterrupt: 

In [ ]:
'''
back up code
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(12)
    
    driver varibles 
 
    # view crusies parent class
    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    # drop down toggle class
    drop_down_xpath = "//button[@class='btn dropdown-toggle btn-outline-black btn-md']"
    drop_down_element = driver.find_element(By.XPATH, drop_down_xpath)
    # drop down individual values
    list_item = driver.find_element(By.XPATH, "//a[@id='bs-select-17-4']")
    # page navigation section
    page_navigation_bar = driver.find_element(By.XPATH, "//*[@class='listing -horizontal']")
    each_page = page_navigation_bar.find_elements(By.TAG_NAME, 'li')
    # initialize varibles to have global scope as its needed for the while loop to work
    page_amount = None
    current_page = None
    # create global list to insert all the list into this one -- remeber to flatten at the end
    global_list_urls = []

    while True:
        if int(current_page.text) > page_amount:
            print("completed all pages")
            break
        else:
            is_page_loaded = "0" not in element.get_attribute("outerText")
            if is_page_loaded:
                print('Page loaded successfully')
                print(element.get_attribute("outerText"))  # Print the outerText of the element
                driver.execute_script("arguments[0].scrollIntoView(true);", drop_down_element)
                
                # Click the dropdown
                driver.execute_script("arguments[0].click();", drop_down_element)
                print('Dropdown clicked successfully')
                
                # Click the individual item selection
                driver.execute_script("arguments[0].click();", list_item) 
                print('Dropdown selection clicked successfully')
                
                # Wait for 20 seconds
                time.sleep(20)
                
                # lets check if the count updated
                if (drop_down_element.get_attribute("title") == "50"):
                    print('Dropdown selection updated successfully')
                    print(drop_down_element.get_attribute("title"))
                    print(driver.current_url)
                    url_list = find_urls(driver)
                    print(url_list) # end
                    
                    # selecting the page navigation
                    page_amount = len(each_page)
                    current_page = page_navigation_bar.find_element(By.XPATH, "//*[@class='active']")
                    print(current_page)
                    
                    

                else:
                    print(drop_down_element.get_attribute("title"))
                    print('Dropdown selection did not update successfully')
                    print(driver.current_url)
            else:
                print('Page did not load successfully')

except WebDriverException as e:
    print('Error loading page:', e)

except ElementNotInteractableException as e:
    print('Element not interactable:', e)

finally:
    # Close the browser
    driver.quit()

'''